In [1]:
import pandas as pd
import tqdm
import openai
import nltk
import os

from pyprojroot import here
from moral_foundations_llms import utils
from textwrap import dedent

In [2]:
comments = pd.read_csv(here('data/aita_comments_Aug17_labels_update_Aug31.csv'))
filled = pd.read_csv(here('data/aita_nan_comments_labels_gpt.csv'))

In [3]:
comments

,id,flair_text_x,comment,score_x,comment_author,comment_time,comment_parent_id,comment_depth,comment_permalink,author_comment_karma,...,sanctity_p,care_sent,fairness_sent,loyalty_sent,authority_sent,sanctity_sent,moral_nonmoral_ratio,f_var,sent_var,comment_label
0,yiplwk,Asshole,"YTA Let me guess, the issues you had staying w...",6282,Tricky-Flamingo-7491,2022-10-31 21:41:58,t3_yiplwk,0,https://reddit.com/r/AmItheAsshole/comments/yi...,148820.0,...,0.079818,-0.074592,0.002464,0.007041,-0.011128,-0.063993,1.691176,0.000045,0.001477,YTA
1,yiplwk,Asshole,**You are guests in their home because you are...,26466,FunkyOrangePenguin,2022-10-31 21:41:03,t3_yiplwk,0,https://reddit.com/r/AmItheAsshole/comments/yi...,387749.0,...,0.079818,-0.074592,0.002464,0.007041,-0.011128,-0.063993,1.691176,0.000045,0.001477,YTA
2,yiplwk,Asshole,YTA and this is a deeply unreasonable request ...,1878,Background-Aioli4709,2022-10-31 21:36:59,t3_yiplwk,0,https://reddit.com/r/AmItheAsshole/comments/yi...,82821.0,...,0.079818,-0.074592,0.002464,0.007041,-0.011128,-0.063993,1.691176,0.000045,0.001477,YTA
3,yiplwk,Asshole,"YTA, and your wife seems to be systematically ...",9640,magstar222,2022-10-31 21:42:03,t3_yiplwk,0,https://reddit.com/r/AmItheAsshole/comments/yi...,104630.0,...,0.079818,-0.074592,0.002464,0.007041,-0.011128,-0.063993,1.691176,0.000045,0.001477,YTA
4,yiplwk,Asshole,YTA - first your wife had issues with your Mom...,11839,RiskBig3301,2022-10-31 21:42:27,t3_yiplwk,0,https://reddit.com/r/AmItheAsshole/comments/yi...,9802.0,...,0.079818,-0.074592,0.002464,0.007041,-0.011128,-0.063993,1.691176,0.000045,0.001477,YTA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531808,122e4xg,NaN,"Honestly, after reading all the comments….\n\n...",2,AshBlackstone78,2023-03-26 12:42:57,t3_122e4xg,0,https://reddit.com/r/AmItheAsshole/comments/12...,23853.0,...,0.082745,-0.077240,-0.000641,-0.002937,-0.019026,-0.049016,1.532468,0.000085,0.001077,NaN
531809,122e4xg,NaN,The ass is the accountant. Make sure you fire...,8,WonderMon,2023-03-26 09:50:49,t3_122e4xg,0,https://reddit.com/r/AmItheAsshole/comments/12...,16847.0,...,0.082745,-0.077240,-0.000641,-0.002937,-0.019026,-0.049016,1.532468,0.000085,0.001077,NaN
531810,122e4xg,NaN,Nta. You’re allowed to keep your own money sep...,7,Even_Satisfaction_37,2023-03-26 10:16:13,t3_122e4xg,0,https://reddit.com/r/AmItheAsshole/comments/12...,1299.0,...,0.082745,-0.077240,-0.000641,-0.002937,-0.019026,-0.049016,1.532468,0.000085,0.001077,NTA
531811,122e4xg,NaN,"ESH. Yes, you've lied to your wife about your ...",4,HammerOn57,2023-03-26 09:18:08,t3_122e4xg,0,https://reddit.com/r/AmItheAsshole/comments/12...,71779.0,...,0.082745,-0.077240,-0.000641,-0.002937,-0.019026,-0.049016,1.532468,0.000085,0.001077,NaN


In [3]:
comments = comments.drop(columns=['flair_text_y'])
comments = comments.rename(
    columns={'flair_text_x': 'flair_text',
             'score_x': 'score_comment',
             'score_y': 'score_submission'})
comments = comments[comments['comment_author'] != 'AutoModerator'] 
comments = comments[comments['comment_author'] != 'AmItheAsshole-ModTeam']

In [4]:
comments = comments.merge(right=filled[['comment_permalink', 'gpt_comment_label']],
               how='left',
               on='comment_permalink')

In [5]:
comments['comment_label'].replace(
    {'NTA (Not the Asshole)': 'NTA',
     'NTA or "Not the Asshole"': 'NTA',
     "ESH (Everyone Sucks Here)": "ESH",
     
     "YWBTA": "YTA",
     "NTAH": "NTA",
     "YTA (You're the Asshole)": "YTA",
     "Label: YTA (You're the Asshole)": "YTA",
     "YTA/ESH": "YTA",
     "NTA/NAH": "NTA",
     "NAH/INFO": "NAH",
     "Label: YTA": "YTA",
     "ESH, but even slight YTA.": "ESH",
     "The comment expresses multiple judgements, including NTA for expecting the friend not to make jokes about the miscarriage, and YTA for expecting the friend to want to be around the baby": "YTA",
     'Based on the comment you provided, it seems to be expressing the judgement of YTA or "You\'re the Asshole". The commenter states that the OP is at fault for not recognizing that the child\'s behavior was normal and for overreacting by involving the MIL.': "YTA",
     "I apologize for the confusion. I am a tool designed to evaluate social media comments and assign them a label based on whether the commenter is seen as the asshole (YTA), not the asshole (NTA), everyone sucks (ESH), no assholes (NAH), not enough information (INFO), or if no specific judgement is being rendered (NONE). I will analyze the comment you provide and assign the appropriate label based on the content of the comment.         1\nNTA to NAH": "NTA",
    }, inplace=True)

In [6]:
comments['comment_label'] = comments['comment_label'].fillna(comments['gpt_comment_label'])
comments.loc[~comments['comment_label'].str.contains('YTA|NTA|INFO|NONE|ESH|INFO|NAH', regex=True), 'comment_label'] = 'NONE'
comments.loc[comments['comment_label'].str.contains('NONE'), 'comment_label'] = 'NONE'
comments.loc[comments['comment_label'].str.contains('INFO'), 'comment_label'] = 'INFO'
comments.loc[comments['comment_label'].str.contains('YTA'), 'comment_label'] = 'YTA'
comments.loc[comments['comment_label'].str.contains('NTA'), 'comment_label'] = 'NTA'
comments.loc[comments['comment_label'].str.contains('ESH'), 'comment_label'] = 'ESH'

In [7]:
comments['comment_label'].value_counts(dropna=False)

NTA     306405
YTA      99228
NONE     82065
ESH      13241
INFO     13138
NAH       9683
Name: comment_label, dtype: int64

In [10]:
comments.to_csv(here('data/aita_all_comments_labeled.csv'), index=False)

/opt/homebrew/Caskroom/miniforge/base/envs/nlp/lib/python3.9/site-packages/pyprojroot/pyprojroot.py:51: UserWarning: Path doesn't exist: /Users/psachdeva/projects/moral_foundations_llms/data/aita_all_comments_labeled.csv
  warnings.warn("Path doesn't exist: {}".format(path))
